In [12]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')
from google.colab import drive
drive.mount('/content/gdrive')
from sklearn.utils import shuffle 
X_train = np.load('/content/gdrive/My Drive/My test/X_LSTM_minTimeSeries_Train.npy')
y_train = np.load('/content/gdrive/My Drive/My test/Y_LSTM_minTimeSeries_Train.npy')
X_test = np.load('/content/gdrive/My Drive/My test/X_LSTM_minTimeSeries_Validation.npy')
y_test = np.load('/content/gdrive/My Drive/My test/Y_LSTM_minTimeSeries_Validation.npy')


X_train1 = X_train[0:75, :, :]
y_train1 = y_train[0:75, :]
X_NonProg = X_train[75: 375, : , :]
y_nonProg = y_train[75:375, :]
X_NonProg, y_nonProg = shuffle(X_NonProg, y_nonProg, random_state = 0)
X_NonProg = X_NonProg[0:75, : , :]
y_nonProg = y_nonProg[0:75, :]

X_train = np.concatenate( (X_train1,  X_NonProg), axis=0)
y_train = np.concatenate((y_train1, y_nonProg), axis=0)
X_test = X_test[:, 0:1590, 0:41]


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [13]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.applications.vgg16 import VGG16

from keras.optimizers import SGD
opt = SGD(lr=0.01, momentum=0.9)

print(X_train.shape, X_test.shape)
input_shape = X_train[0].shape
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

modelC2D = Sequential()
modelC2D.add(Dense(64, activation='relu'))
modelC2D.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(X_train.shape[1], X_train.shape[2], 1)))
#modelC2D.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization(mode = 0, trainable = False))
modelC2D.add(MaxPooling2D(pool_size=(2, 2)))
modelC2D.add(Dropout(0.15))
modelC2D.add(Flatten())
modelC2D.add(Dense(128, activation='relu'))
modelC2D.add(Dropout(0.2))
modelC2D.add(Dense(64, activation='relu'))
modelC2D.add(Dense(1, activation='sigmoid'))

modelC2D.compile(loss=keras.losses.mean_squared_error,
              optimizer='sgd',#keras.optimizers.Adadelta(),
              metrics=['accuracy'])

modelC2D.fit(X_train, y_train,
          batch_size=10,
          epochs=2,
          verbose=1,
          validation_data = (X_test, y_test))


(150, 1590, 41) (92, 1590, 41)
Instructions for updating:
Use tf.cast instead.
Train on 150 samples, validate on 92 samples
Epoch 1/2
150/150 [==============================] - 113s 750ms/step - loss: 0.5193 - acc: 0.4800 - val_loss: 0.7717 - val_acc: 0.2283
Epoch 2/2
150/150 [==============================] - 100s 669ms/step - loss: 0.5133 - acc: 0.4867 - val_loss: 0.7717 - val_acc: 0.2283


In [15]:
from sklearn.metrics import confusion_matrix
y_pred1 = modelC2D.predict(X_test)
for i in range(0,len(y_pred1)):
    if y_pred1[i] > 0.5:
        y_pred1[i] = 1
    else:
        y_pred1[i] = 0
confusion_matrix(y_test, y_pred1)

array([[ 0, 71],
       [ 0, 21]])